In [ ]:
import jax
from jax import config
config.update("jax_enable_x64", True)

from pathlib import Path

from matplotlib import pyplot as plt
import numpy as np

from viperleed_jax.tensor_calculator import benchmark_calculator

jax.devices()

In [2]:
%matplotlib inline

In [3]:
from viperleed_jax.from_state import calculator_from_state
from viperleed_jax.parameter_space import ParameterSpace
from viperleed_jax.parameters.geo_parameters import GeoParamBound
from viperleed_jax.parameters.vib_parameters import VibParamBound
from viperleed_jax.parameters.v0r_parameters import V0rParamBound

In [4]:
from viperleed.calc.files.displacements import readDISPLACEMENTS
from viperleed.calc import LOGGER as logger
from viperleed.calc.files.phaseshifts import readPHASESHIFTS
from viperleed.calc.files.iorfactor import beamlist_to_array

In [ ]:
input_base_path = Path('../tests/test_data/Cu_111')
large_file_path = Path('../large_files/')

origin_path = input_base_path /'dynamic_l_max'
tensor_path = large_file_path / 'Cu_111' / 'dynamic_l_max' /'Tensors' / 'Tensors_001.zip'

# origin_path = input_base_path /'fixed_l_max'
# tensor_path = large_file_path / 'Cu_111' / 'fixed_l_max' /'Tensors' / 'Tensors_001.zip'

calculator, slab, rpars, ref_data, phaseshifts = calculator_from_state(origin_path,
                                                                       tensor_path,
                                                                       l_max=10,
                                                                       batch=1000,
                                                                       recalculate_ref_t_matrices=False)

In [ ]:
parameter_space = ParameterSpace(slab)

## GEOMETRY
# Fix layer 0
parameter_space.geo_params.fix_layer(1, z_offset=0.)
parameter_space.geo_params.fix_layer(2, z_offset=0.)
parameter_space.geo_params.fix_layer(3, z_offset=0.)
parameter_space.geo_params.fix_layer(4, z_offset=0.)

# symmetry constrained xyz movements ± 0.15 A for layer 2
for param in [p for p in parameter_space.geo_params.terminal_params if p.bound is None]:
    param.set_bound(GeoParamBound(-0.05, + 0.05))

## VIBRATIONS
# fix *_def sites (O_def, Fe_def)
for param in [p for p in parameter_space.vib_params.terminal_params if p.site_element.site.endswith('_def')]:
    parameter_space.vib_params.fix_site_element(param.site_element, None) # None fixes to the default value


# the rest can vary ± 0.05 A
for param in [p for p in parameter_space.vib_params.terminal_params if p.site_element.site.endswith('_surf')]:
    param.set_bound(VibParamBound(-0.05, + 0.05))

## CHEMISTRY
# no free parameters
parameter_space.occ_params.remove_remaining_vacancies()

# V0R
# set ± 2 eV
parameter_space.v0r_param.set_bound(V0rParamBound(-2., +2.))

calculator.set_parameter_space(parameter_space)

In [9]:
amps = calculator.jit_delta_amplitude((0.5, 0.5, 0.5, 0.5, 0.5))